# Hunting Outliers

# TODO:
1. Use sklearn or other tools to find outliers for all the features. Finding smallest and largest values manually is not so efficient!
2. Use z-score to find outliers

# Introduction:
In this notebook, we will hunt outliers

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from pathlib import Path
from IPython.display import display
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgbm
import catboost

In [2]:
from warnings import filterwarnings
filterwarnings("ignore")

# Loading Data

In [51]:
# setting a base path variable for easy access
BASE_PATH = Path("/kaggle/input/playground-series-s3e6")
train = pd.read_csv(BASE_PATH / "train.csv").drop(columns=["id"])

test = pd.read_csv(BASE_PATH / "test.csv")
# we need the test id column to make the submission
test_idx = test.id
test = test.drop(columns=["id"])

original = pd.read_csv("/kaggle/input/paris-housing-price-prediction/ParisHousing.csv")

In [52]:
# features presence check
all(original.columns == train.columns)

True

In [53]:
all_datasets = {"train": train, "test": test,"original": original}

# Little preprocessing

In [54]:
def preprocess(train, original):
    X = train.drop(columns="price")
    y = train.price
    X_org = original.drop(columns="price")
    y_org = original.price
    
    return X, y, X_org, y_org

# Train models and check score

In [55]:
def cross_validate(X, y, X_org, y_org, model, model_verbose):
    N_FOLDS = 5
    cv_scores = np.zeros(N_FOLDS)
    feature_importances_all_folds = np.zeros(shape=(N_FOLDS, len(X.columns)))
    
    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=1337)
    
    for fold_num, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
#         # As has been shown in the previous competitions that
#         # train on the combined(competition + original) dataset
#         # but evaluate only on competition dataset
#         X_train = pd.concat([X_train, X_org], axis=0)
#         y_train = pd.concat([y_train, y_org], axis=0)
        
        if model=="XGBoost":
            model = xgb.XGBRegressor()
            
        elif model=="LightGBM":
            model = lgbm.LGBMRegressor()
            
        elif model=="CatBoost":
            model = catboost.CatBoostRegressor(eval_metric="RMSE")
        
        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  early_stopping_rounds=50,
                 verbose=model_verbose)
        
        y_preds = model.predict(X_val)
        
        # to calculate rmse instead of mse, we set squared=False
        rmse = mean_squared_error(y_val, y_preds, squared=False)
        cv_scores[fold_num] = rmse        
        print(f"Fold {fold_num} \t RMSE: {rmse}")
        
    avg_rmse = np.mean(cv_scores)
    print(f"AVG RMSE: {avg_rmse}")

In [56]:
def try_new_changes(X, y, X_org, y_org):    
    models = ["XGBoost", "LightGBM", "CatBoost"]
    
    for model in models:
        print(f"\n{'-'*30} {model} {'-'*30}")
        
        verbose = False
        if model=="LightGBM":
            verbose = -1
            
        cross_validate(X, y, X_org, y_org, model=model, model_verbose=verbose)

### Let's run it and set a baseline

In [57]:
try_new_changes(*preprocess(train, original))


------------------------------ XGBoost ------------------------------
Fold 0 	 RMSE: 173732.7748770942
Fold 1 	 RMSE: 139404.0495448856
Fold 2 	 RMSE: 97092.11937356321
Fold 3 	 RMSE: 242539.94010373257
Fold 4 	 RMSE: 148954.69073388446
AVG RMSE: 160344.71492663203

------------------------------ LightGBM ------------------------------
Fold 0 	 RMSE: 169410.45032032923
Fold 1 	 RMSE: 132249.73024736258
Fold 2 	 RMSE: 133168.19074826475
Fold 3 	 RMSE: 251895.71308501367
Fold 4 	 RMSE: 159952.15709558927
AVG RMSE: 169335.2482993119

------------------------------ CatBoost ------------------------------
Fold 0 	 RMSE: 172736.27950287735
Fold 1 	 RMSE: 130786.56625920956
Fold 2 	 RMSE: 117967.48104400415
Fold 3 	 RMSE: 240842.8287274359
Fold 4 	 RMSE: 146499.10749395363
AVG RMSE: 161766.45260549613


# Prelimiary Data Analysis to see if we can spot anything weird

In [14]:
# pd.concat([train.isnull().sum().rename("Missing In Train"),
#           test.isnull().sum().rename("Missing in Test"),
#           original.isnull().sum().rename("Missing in Original")], axis=1)

pd.concat([dataset.isnull().sum().rename(f"Missing in {dataset_name}") 
               for dataset_name, dataset in all_datasets.items()],
         axis=1)

,Missing in train,Missing in test,Missing in original
squareMeters,0,0.0,0
numberOfRooms,0,0.0,0
hasYard,0,0.0,0
hasPool,0,0.0,0
floors,0,0.0,0
cityCode,0,0.0,0
cityPartRange,0,0.0,0
numPrevOwners,0,0.0,0
made,0,0.0,0
isNewBuilt,0,0.0,0


In [15]:
# [1] + \
# [x for x in range(2, 5)]

[1, 2, 3, 4]

In [17]:
# pd.concat([train.dtypes.rename("Data Type"),
#           train.nunique().rename("Train UniqueValues"),
#           test.nunique().rename("Test UniqueValues"),
#           original.nunique().rename("Original UniqueValues")], axis=1)\
#             .sort_values(by="Train UniqueValues")

pd.concat([train.dtypes.rename("Data Type")] + \
          [dataset.nunique().rename(f"{dataset_name} UniqueValues") for dataset_name, dataset in all_datasets.items()],
          axis=1).sort_values(by="train UniqueValues")

,Data Type,train UniqueValues,test UniqueValues,original UniqueValues
hasYard,int64,2,2.0,2
hasPool,int64,2,2.0,2
hasStorageRoom,int64,2,2.0,2
isNewBuilt,int64,2,2.0,2
hasStormProtector,int64,2,2.0,2
cityPartRange,int64,10,10.0,10
numPrevOwners,int64,10,10.0,10
hasGuestRoom,int64,11,11.0,11
made,int64,33,32.0,32
numberOfRooms,int64,100,100.0,100


### INSIGHTS:
1. **made** represents the year probably in which the house was made. This feature in train contains 33 values but only does 32 in test and original. Let's inveestigate that first.

# Outliers in "made" feature:
**made** represents the year probably in which the house was made. This feature in train contains 33 values but only does 32 in test and original. Let's inveestigate!

In [31]:
# Let's first verify that the "made" values for test and original are the same 32
test.made.unique().sort() == original.made.unique().sort()

True

In [32]:
# Let's find the values that's only present in train
set(train.made.unique()) - set(test.made.unique())

{10000}

### Thoughts:
This is definitely an anomalous value as 10000 makes no sense for a year.
Let's see which rows contain this value.

In [33]:
train[train.made == 10000]

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
2113,68038,41,0,0,54,87120,3,6,10000,1,1,6537,6304,366,0,0,6807415.1
3608,80062,81,1,0,35,67157,9,4,10000,0,1,732,6475,758,0,4,8007951.1
19124,80062,52,0,0,84,67099,9,4,10000,0,0,7677,5017,148,0,4,8007951.1
19748,80062,58,0,1,86,40408,7,8,10000,0,0,7059,7307,287,0,2,8007951.1
21400,80062,78,0,0,84,59457,4,7,10000,1,0,6382,9507,298,1,4,8007951.1


In [34]:
# lets see if there are outliers in test or original
display(test[test.made > 2021])
display(original[original.made > 2021])

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom


,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price


### Thoughts:
No outliers in test and original, but definitely in train. We definitely need to fix this by using some valid value. Maybe the most recent year one?

## Fixing
1. using the most recent year value to replace the outlier
    - Didn't work, makes some models act worse
2. Let's try dropping these rows instead

In [35]:
outlier_indices = train[train.made==10000].index
# --------------- idea #1 -------------------
# train.loc[outlier_indices, "made"] = 2021

# -------------- idea #2 --------------------
train.drop(outlier_indices, inplace=True)

In [37]:
try_new_changes(*preprocess(train, original))


------------------------------ XGBoost ------------------------------
Fold 0 	 RMSE: 45674.38886208899
Fold 1 	 RMSE: 231709.36857522716
Fold 2 	 RMSE: 161041.3939813482
Fold 3 	 RMSE: 188465.777224332
Fold 4 	 RMSE: 161757.85324284332
AVG RMSE: 157729.75637716794

------------------------------ LightGBM ------------------------------
Fold 0 	 RMSE: 78220.3786947944
Fold 1 	 RMSE: 290665.1718351153
Fold 2 	 RMSE: 167473.49470280512
Fold 3 	 RMSE: 179306.65190895365
Fold 4 	 RMSE: 155404.10962694066
AVG RMSE: 174213.96135372185

------------------------------ CatBoost ------------------------------
Fold 0 	 RMSE: 76366.73146243462
Fold 1 	 RMSE: 303633.0513196458
Fold 2 	 RMSE: 167827.87027511184
Fold 3 	 RMSE: 166269.56544490522
Fold 4 	 RMSE: 154120.8214504736
AVG RMSE: 173643.6079905142


# Outliers in "garage" feature
The garage feature contains value for how big the garage is. It should definitely be in all cases a fraction of the total house size.

But let's see if there are instances where the garage size is greater than the actual house size, which will definitely be an outlier.

In [58]:
for dataset_name, dataset in all_datasets.items():
    print(f"{'-'*30} {dataset_name.upper()} OUTLIERS {'-'*30}")
    print(f"Number of rows with garage size greater than house size: " , len(dataset[dataset.garage > dataset.squareMeters]))
    print()
    
# train[train.garage > train.squareMeters]

------------------------------ TRAIN OUTLIERS ------------------------------
Number of rows with garage size greater than house size:  261

------------------------------ TEST OUTLIERS ------------------------------
Number of rows with garage size greater than house size:  186

------------------------------ ORIGINAL OUTLIERS ------------------------------
Number of rows with garage size greater than house size:  59



## INSIGHTS:
Welp, that's a LOT of outliers, not just in train, but also in test and original. and most importancely in test!!

## What should we do?
Okay, since there are quite a lot these outliers in test set as well, we should come up with a strat to address this issue. Dropping rows with outliers it not an option anymore!

### Solution:
One thing that I'm thinking is that we should find the average garage to squareMeters ratio and use that to fix the garage sizes for these outlier values.

In [59]:
avg_garage_to_house_size_ratio = np.mean(train.garage / train.squareMeters)
avg_garage_to_house_size_ratio * 100

6.216653585508132

## Hmm,
So on average, a garage takes only 6.21 percentance of a house's whole area, which makes complete sense.

## Fixing
To fix the outliers, we should multiply this mean value with squareMeters for the outlier records only and replace the value in garage feature with the new value.

In [60]:
# outliers = train.garage > train.squareMeters
# before = train[outliers]["garage"]
# outlier_indices = np.array(train[outliers].index)
# train["garage"].iloc[outlier_indices] = (train.iloc[outlier_indices]["squareMeters"] * avg_garage_to_house_size_ratio).astype("int64").to_numpy()
# after = train.iloc[outlier_indices]["garage"]

for dataset_name, dataset in all_datasets.items():
    print(f"{'-'*30} {dataset_name.upper()} Ooutliers Resolution {'-'*30}")
    garage_outliers = dataset.garage > dataset.squareMeters
    outlier_indices = dataset[garage_outliers].index
    print("Before: ")
    print(f"\tNumber of rows with garage size greater than house size: " , len(dataset[garage_outliers]))
    print(f"\tAvg garage size of outliers: ", np.mean(dataset[garage_outliers].garage))
    
    dataset.loc[outlier_indices, "garage"] = (dataset.loc[outlier_indices].squareMeters * \
                                                  avg_garage_to_house_size_ratio).astype("int64").to_numpy()
    print("After: ")
    print(f"\tNumber of rows with garage size greater than house size: " , len(dataset[dataset.garage > dataset.squareMeters]))
    print(f"\tAvg garage size of outliers: ", np.mean(dataset[garage_outliers].garage))
    print("\n")

------------------------------ TRAIN Ooutliers Resolution ------------------------------
Before: 
	Number of rows with garage size greater than house size:  261
	Avg garage size of outliers:  753.0459770114943
After: 
	Number of rows with garage size greater than house size:  0
	Avg garage size of outliers:  27.50191570881226


------------------------------ TEST Ooutliers Resolution ------------------------------
Before: 
	Number of rows with garage size greater than house size:  186
	Avg garage size of outliers:  746.6451612903226
After: 
	Number of rows with garage size greater than house size:  0
	Avg garage size of outliers:  28.333333333333332


------------------------------ ORIGINAL Ooutliers Resolution ------------------------------
Before: 
	Number of rows with garage size greater than house size:  59
	Avg garage size of outliers:  729.0508474576271
After: 
	Number of rows with garage size greater than house size:  0
	Avg garage size of outliers:  24.983050847457626




### Thoughts:
Fixed, Yes! Hopefully it will improve performace! InshaAllah!

In [62]:
try_new_changes(*preprocess(train, original))


------------------------------ XGBoost ------------------------------
Fold 0 	 RMSE: 173732.7748770942
Fold 1 	 RMSE: 139459.75059841236
Fold 2 	 RMSE: 97564.53836806856
Fold 3 	 RMSE: 244953.38625612677
Fold 4 	 RMSE: 149463.52626601956
AVG RMSE: 161034.7952731443

------------------------------ LightGBM ------------------------------
Fold 0 	 RMSE: 169410.4071199048
Fold 1 	 RMSE: 132072.24552027133
Fold 2 	 RMSE: 132498.14873211033
Fold 3 	 RMSE: 253344.33062868376
Fold 4 	 RMSE: 158704.96898157502
AVG RMSE: 169206.02019650902

------------------------------ CatBoost ------------------------------
Fold 0 	 RMSE: 173190.40588546768
Fold 1 	 RMSE: 128527.31634171898
Fold 2 	 RMSE: 121979.98555910251
Fold 3 	 RMSE: 240135.40312722273
Fold 4 	 RMSE: 147504.7861500053
AVG RMSE: 162267.57941270343


# Outliers in Attic feature
Attic represents the attic size of the house in sq meters. Let's see if like garage it contains any values that are greater than the actual hosue size!

## Scaning:

In [17]:
for dataset_name, dataset in all_datasets.items():
    print(f"{'-'*30} {dataset_name.upper()} Attic Outliers Scan {'-'*30}")
    attic_outliers = dataset.attic > dataset.squareMeters
    outlier_indices = dataset[attic_outliers].index
    print(f"\tNumber of rows with attic size greater than house size: " , len(dataset[attic_outliers]))
    print("\n")

------------------------------ TRAIN Attic Outliers Scan ------------------------------
	Number of rows with attic size greater than house size:  1681


------------------------------ TEST Attic Outliers Scan ------------------------------
	Number of rows with attic size greater than house size:  1162


------------------------------ ORIGINAL Attic Outliers Scan ------------------------------
	Number of rows with attic size greater than house size:  531




### Let's check the average attic to house size ratio

In [20]:
avg_attic_to_house_size_ratio = np.mean(train.attic / train.squareMeters)
avg_attic_to_house_size_ratio

0.5579126968257411

#### INSIGHTS:
On average an attic takes up 55.7% of the house, which kinda does make sense

In [23]:
attic_outliers = train.attic >= train.squareMeters
print("TRAIN: Number of rows with attic size greater than the house size: ", len(train[attic_outliers]))
train[attic]

Number of rows with attic size greater than the house size:  1681


# More coming soon!